# `views.py` 복습

## create 함수 본문
```python
from django.shortcuts import render, redirect
from .forms import ArticleForm, CommentForm
from django.contrib.auth.decorators import login_required

@login_required
def create(request):
    if request.method == 'POST':
        form = ArticleForm(request.POST)
        if form.is_valid():
            article = form.save(commit=False)
            article.user = request.user
            article.save()
            return redirect('articles:index')

    else:
        form = ArticleForm()
    context = {
        'form': form,
    }
    return render(request, 'create.html', context)
```

### `@login_required`
Django에서 사용되는 뷰 데코레이터로, 해당 뷰에 접근하기 전에 로그인이 되어 있는지 확인하는 기능을 함

- redirect_field_name: 로그인하지 않은 경우 리다이렉트 시 전달할 쿼리 문자열의 파라미터명 (기본값: 'next')

#### 쿼리 문자열과 파라미터란?

- 쿼리 문자열은 URL에서 물음표 `?` 뒤에 위치하며, 여러 개의 `키=값` 형태로 구성
- 예시 : `/accounts/login/?next=/orders/`

#### redirect_field_name의 역할
1. 비로그인 사용자가 접근 시 :
    로그인 페이지로 리다이렉트하면서 원래 이동하려던 url을 next에 저장
2. 로그인 성공 시 :
    next에 저장된 url로 리다이렉트(원래 가고자했던 url로 이동)

#### 본문기준 url 동작 흐름

1. 비로그인 사용자가 /create/에 접근
    > /accounts/login/?next=/create/
    - next=/create/는 로그인 후 복귀할 경로
2. 로그인 성공 후
    > /create/ (원래 접근하려던 경로로 복귀)
3. 성공적으로 글 작성 시
    > 리다이렉트 대상인 articles:index 경로

### `else: form = ArticleForm()`
먼저 빈 폼을 사용자에게 제공하는 else 구문이 먼저 실행

### form에 글을 쓴 뒤
submit을 누르면 POST 요청으로 인해 if문 True가 실행
```python
if request.method == 'POST':
        form = ArticleForm(request.POST)
        if form.is_valid(): # 유효성 검사
            article = form.save(commit=False) # 현재 제출된 form에는 article을 작성한 user_id가 없으므로 대기상태 만듬
            article.user = request.user # article의 user_id 컬럼에 request 들어온 user_id 값 할당
            article.save() # title, content, user_id 모두 저장
            return redirect('articles:index')
```

##### form.is_valid() 통과 후 주의점
`ArticleForm`에서 `user` 필드를 제외하고 `form.is_valid()`를 통과한 후 바로 `article.save()`를 호출하면 DB 저장 단계에서 저장 실패가 발생
- Django의 ForeignKey 기본 설정
    - ForeignKey 필드는 `user = models.ForeignKey(User, on_delete=models.CASCADE)` 라고만 작성해도 `null=False` 조건 적용

### Django ForeignKey의 on_delete 옵션
Django의 ForeignKey는 부모 객체가 삭제될 때 자식 객체의 처리 방식을 on_delete 옵션으로 지정
- 게시글(부모객체)이 삭제되면 댓글들(자식객체)은 어떻게 처리할 것인지

#### 1. CASCADE
부모 객체가 삭제되면 연결된 자식 객체도 함께 삭제

```python
class Article(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)
```

사용자가 삭제되면 해당 사용자가 작성한 모든 Article이 삭제

## login 함수 본문
```python
from django.shortcuts import render, redirect
from .forms import CustomAuthenticationForm
from django.contrib.auth import login as auth_login

def login(request):
    if request.method == 'POST':
        form = CustomAuthenticationForm(request, request.POST)
        if form.is_valid():
            user = form.get_user() # 로그인한 user 객체 가져옴
            auth_login(request, user)
            return redirect('posts:index')
    else:
        form = CustomAuthenticationForm()
    
    context = {
        'form': form,
    }
    return render(request, 'login.html', context)
```

## Django의 login
- 사용자 인증을 처리하는 함수
> views.py에서 함수 작성시 `def login()`으로 작명해서 `as auth_login`으로 표기

### `else: form = CustomAuthenticationForm()`
log in 폼을 보여준다

### `if`가 True일 때,
`request`로 들어온 모든 정보와 폼에 `request.POST`로 들어온 정보를 둘다 가져온다

- 그리고 `form`이 유효하다면 (코드 작성 시 자주 놓치는 부분 **주의!!**))
> `form`에 user정보를 가져와서 할당

### `auth_login(request, user)`
auth_login(원래는 django의 login 뷰)으로 request와 user를 대조/그 외 여러 인증작업 시행
- 대조하여 불일치 시, 다시 빈 폼의 login.html로 리다이렉트 하는 등 세션 기반 로그인 처리를 수행

## logout 함수 본문
```python
from django.shortcuts import render, redirect
from django.contrib.auth import logout as auth_logout

def logout(request):
    auth_logout(request)
    return redirect('posts:index')
```

## django의 logout
- session cookie 내에 저장된 사용자 로그인 정보를 삭제처리
> views.py에서 함수 작성시 `def logout()`으로 작명해서 `as auth_logout`으로 표기

### `auth_logout(request)`
단순히 해당 session 데이터만 삭제해줌